In [1]:
from math import floor, log2
from sympy import prime
import random

In [52]:
def gcd(a, b): # b > a
    if a>b:
        temp = a
        a = b
        b = temp
    q_0 = 0
    a_0 = a
    b_0 = b
    ### extended gcd ###
    s_0 = 1
    s_1 = 0
    t_0 = 0
    t_1 = 1
    ### extended gcd ###
    while True:
        q_0 = 0
        while b_0>=a_0: #  and a_0!=0
            b_0 -= a_0
            q_0+=1
        ### extended gcd ###
        temp_s = s_1
        s_1 = s_0-q_0*s_1
        s_0 = temp_s
        temp_t = t_1
        t_1 = t_0-q_0*t_1
        t_0 = temp_t
        # print(t_0, t_1, s_0, s_1)
        ### extended gcd ###
        if (q_0==0 or b_0==0): break
        rem = b_0
        b_0 = a_0
        a_0 = rem
        if (a_0==1):
            temp_s = s_1
            s_1 = s_0-q_0*s_1
            s_0 = temp_s
            temp_t = t_1
            t_1 = t_0-q_0*t_1
            t_0 = temp_t
            break
    return a_0, t_0, s_0
def lcm(a,b):
    return abs(a*b)//gcd(a,b)[0]
ascii32bit = "abcdefghijklmnopqrstuvwxyz.,!?' "
def ascii32bit_encode(s: str):
    acc = 0
    i = 0
    for c in s:
        acc+=(ascii32bit.index(c) if c in ascii32bit else 31)<<i
        i+=5
    return acc
# edge case where last char has leading zeros, need to adjust bit shift offset accordingly
def ascii32bit_decode(s: int):
    acc = s
    ans = ""
    n = round(log2(s)) - 5
    n = n+(5-n%5) if n%5!=0 else n
    for i in range(n, -1, -5):
        temp = acc >> i
        temp2 = temp << i
        ans=ascii32bit[temp]+ans
        acc -= temp2
    return ans
def fast_mod_exp(b, exp, m):
    res = 1
    while exp > 1:
        if exp & 1:
            res = (res * b) % m
        b = b ** 2 % m
        exp >>= 1
    return (b * res) % m
# ported from https://stackoverflow.com/questions/2921406/calculate-primes-p-and-q-from-private-exponent-d-public-exponent-e-and-the
# given d e and n, find p and q
def pq_from_den(d, e, n):
    k = d*e-1
    if (k%2==0):
        r = k
        t = 0
        while (r%2==0):
            r = r >> 1
            t+=1
        success = False
        y = -1
        for i in range(1,101):
            g = -1
            g = random.randrange(0,n)
            y = fast_mod_exp(g,r,n)
            if y==1 or y==n-1:
                continue
            for j in range(1, t):
                x = fast_mod_exp(y,2,n)
                if x==1:
                    success = True
                    break
                if x==n-1:
                    continue
                y = x
            x = fast_mod_exp(y, 2, n)
            if (x==1):
                success=True
                break
        if (success):
            p = gcd(y-1, n)[0]
            q = n//p
            return p, q
        else:
            assert False, "not found"
    else:
        assert False, "not found"

In [36]:
def safe(): # wiener attack
    p = 9903148612062279695184974394942430814302817287340215681696223192481788630483586667090617147331535537896655460738209367624045422088081722029580391041035523 # prime(18000)
    q = 7763338934602697279748332648086085192267317357016587041443338139740907849594395036920888068586614237923957000946534416713286157525947261857424168392169367 # prime(10000)
    n = 76881499195179758719860576028129204256330463041538761885870902922146174453818918859116923311186372704033550511757618765150988598753482039061299751106213131549269527605251067558555225306491166622692816670944375066132653303087101319120517392175340003297430678476513934572673535247748275637537396895236079423941
    lambda_n = lcm(q-1,p-1)
    e =  60622565810056765588076682990247867931467974177173929430876799924141527547159343311956425490167737740240046629439818049884054674496400421837121395839395671068760025748314749504507293154243658839233494420713870324550776601235709721477057208088323328590472437615906346795333144247497340280153973078100120070771 #prime(2598) # 1<e<lambda_n and gcd(e, lambda_n)=1
    _, s, t = gcd(e, lambda_n)
    d = s if s>=0 else t

    public_key = (n, e)
    private_key = (d)
    other_private = (p, q, lambda_n)

    M = "howdy! i'm flowey! flowey the flower!"
    m = ascii32bit_encode(M) # not padded for simplicity
    c = fast_mod_exp(m,e,n)
    d_m = fast_mod_exp(c,d,n)
    d_M = ascii32bit_decode(d_m)
    return d_M

In [56]:
def safe2(): # wiener attack
    p = 10553161238686887161967409946215517718927097707283446248444147056478295701628838694393052399921524231974833455905429121406898275473106957114920651752668943 # prime(18000)
    q = 8612291197242305949361127097989333930575006159863404770671237117501284961531631303777294502095685085474980726947916288919190873836152132786162383573254907 # prime(10000)
    n = 90886897639021787897045312081059096830632912244785231795090265536247326168553621771976278598828533782362583293811148009214304601725792155420398203834801032929475677124187729195263977143640871617218069503692234334997728560490127434657328608028190920343459912921255279113199654122046519394645371826790921253301
    lambda_n = lcm(q-1,p-1)
    e =  83835725165889269222562491553708810684281834744477974826262570700359622812128997882770831047602710683313120057056522647188687755995559965065812990032959290033267644948408091846617420530077206949775273796046377471516125919379530844610917160683409638808632641541296248915332818480164498319742769877817729596967 #prime(2598) # 1<e<lambda_n and gcd(e, lambda_n)=1
    _, s, t = gcd(e, lambda_n)
    d = s if s>=0 else t

    public_key = (n, e)
    private_key = (d)
    other_private = (p, q, lambda_n)

    M = "howdy! i'm flowey! flowey the flower!"
    m = ascii32bit_encode(M) # not padded for simplicity
    c = fast_mod_exp(m,e,n)
    d_m = fast_mod_exp(c,d,n)
    d_M = ascii32bit_decode(d_m)
    return d_M

In [48]:
from sympy.solvers import solve
from sympy import Symbol
from sympy.ntheory.continued_fraction import continued_fraction_convergents
from sympy import Rational
from sympy.ntheory import continued_fraction
def attack(e,n):
    cf = continued_fraction(Rational(e,n))
    for c in list(continued_fraction_convergents(cf)):
        k = c.p
        d = c.q
        if (k==0 or d%2==0 or e*d%k!=1):
            continue
        phi = (e*d - 1)/k
        x = Symbol('x')
        print(d, solve(x**2 - ((n-phi)+1)*x + n, x))

In [49]:
attack(e,n)

30588114826971426600657838192605209153414751918953694408310787095242119877703 [0.5 - 9.53346199651636e+153*I, 0.5 + 9.53346199651636e+153*I]
